# Create an interface between two materials with minimal strain

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).

<h2 style="color:green">Usage</h2>

0. Make sure to select Input Materials
1. Execute "Run first: ..." cell below to load Input Materials into the current kernel
2. Set Input Parameters (e.g. `MILLER_INDICES`, `THICKNESS`, `MAX_AREA`) below or use the default values
3. Click "Run" > "Run All" to run all cells
4. Wait for the run to complete (depending on the area, it can take 1-2 min or more). Scroll down to view cell results.
5. Review the strain plot and modify its parameters as needed

## Methodology

The following happens in the script below:

1. Create slabs for each input material. The materials data is passed in from and back to the web application according to this description (TBA).
   We assume that two input materials are either in bulk form (e.g. Ni crystal) or layered (e.g. graphene). 
   
   We construct the interface along the Z-axis. The material corresponding to the bottom of the interface is referred to as the "**substrate**", and the top - as the "**layer**". 

2. Perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered, that can be configured by editing the cells below.

3. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. We create the corresponding supercells and place them at a specified distance from each other (note no shift is performed currently).


<h2 style="color:red">Run first: load input materials in current kernel</h2>


In [1]:
import json
materials_in = []
for file in ["Ni.json", "Gr.json"]:
    with open(file, "r") as f:
        data = f.read()
        materials_in.append(json.loads(data))

## 1. Set Input Parameters

### 1.1. Select Substrate and Layer from Input Materials

In [2]:
SUBSTRATE_PARAMETERS = {
    "MATERIAL_INDEX": 0,  # the index of the material in the materials_in list
    "MILLER_INDICES": (1, 1, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 3,  # in layers
}

LAYER_PARAMETERS = {
    "MATERIAL_INDEX": 1, # the index of the material in the materials_in list
    "MILLER_INDICES": (0, 0, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 1,  # in layers
}

### 1.2. Set Interface Parameters

The distance between layer and substrate and maximum area to consider when matching.


In [3]:
INTERFACE_PARAMETERS = {
    "DISTANCE_Z": 3.0, # in Angstroms
    "MAX_AREA": 50, # in Angstroms^2
}

### 1.3. Set Algorithm Parameters

In [4]:
ZSL_PARAMETERS = {
    "MAX_AREA": INTERFACE_PARAMETERS["MAX_AREA"],  # The area to consider in Angstrom^2
    "MAX_AREA_TOL": 0.09,  # The area within this tolerance is considered equal
    "MAX_LENGTH_TOL": 0.03,  # supercell lattice vectors lengths within this tolerance are considered equal
    "MAX_ANGLE_TOL": 0.01,  # supercell lattice angles within this tolerance are considered equal
    "STRAIN_TOL": 10e-6,  # strains within this tolerance are considered equal
}

## 2. Install Packages

In [5]:
!python --version
!pip list
#!pip install pymatgen==2024.2.8 ase==3.22.1 nbformat==5.9.2 ipykernel==6.29.2 matgl==0.9.2 nglview==3.1.1

Python 3.10.12
Package                   Version
------------------------- ---------------
aiohttp                   3.9.3
aiosignal                 1.3.1
anyio                     4.2.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
ase                       3.22.1
asttokens                 2.4.1
async-lru                 2.0.4
async-timeout             4.0.3
attrs                     23.2.0
Babel                     2.14.0
beautifulsoup4            4.12.3
bleach                    6.1.0
certifi                   2024.2.2
cffi                      1.16.0
charset-normalizer        3.3.2
comm                      0.2.1
contourpy                 1.2.0
cycler                    0.12.1
debugpy                   1.8.1
decorator                 5.1.1
defusedxml                0.7.1
dgl                       2.0.0
exceptiongroup            1.2.0
executing                 2.0.1
fastjsonschema            2.19.1
f

## 3. Create interfaces

### 3.1. Extract Interfaces and Terminations

Extract all possible layer/substrate supercell combinations within the maximum area including different terminations.

In [6]:
from src.pymatgen_coherent_interface_builder import CoherentInterfaceBuilder, ZSLGenerator
from src.utils import to_pymatgen

if "materials_in" in globals():
    pymatgen_materials = [to_pymatgen(item) for item in materials_in]
for material in pymatgen_materials:
    print(material, "\n")


def create_interfaces(settings):
    print("Creating interfaces...")
    zsl = ZSLGenerator(
        max_area_ratio_tol=settings["ZSL_PARAMETERS"]["MAX_AREA_TOL"],
        max_area=settings["ZSL_PARAMETERS"]["MAX_AREA"],
        max_length_tol=settings["ZSL_PARAMETERS"]["MAX_LENGTH_TOL"],
        max_angle_tol=settings["ZSL_PARAMETERS"]["MAX_ANGLE_TOL"],
    )

    cib = CoherentInterfaceBuilder(
        substrate_structure=pymatgen_materials[settings["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"]],
        film_structure=pymatgen_materials[settings["LAYER_PARAMETERS"]["MATERIAL_INDEX"]],
        substrate_miller=settings["SUBSTRATE_PARAMETERS"]["MILLER_INDICES"],
        film_miller=settings["LAYER_PARAMETERS"]["MILLER_INDICES"],
        zslgen=zsl,
        strain_tol=settings["ZSL_PARAMETERS"]["STRAIN_TOL"],
    )

    # Find terminations
    cib._find_terminations()
    terminations = cib.terminations

    # Create interfaces for each termination
    interfaces = {}
    for termination in terminations:
        interfaces[termination] = []
        for interface in cib.get_interfaces(
            termination,
            gap=settings["INTERFACE_PARAMETERS"]["DISTANCE_Z"],
            film_thickness=settings["LAYER_PARAMETERS"]["THICKNESS"],
            substrate_thickness=settings["SUBSTRATE_PARAMETERS"]["THICKNESS"],
            in_layers=True,
        ):
            # Wrap atoms to unit cell
            interface["interface"].make_supercell((1,1,1), to_unit_cell=True)
            interfaces[termination].append(interface)
    return interfaces, terminations


interfaces, terminations = create_interfaces(
    settings={
        "SUBSTRATE_PARAMETERS": SUBSTRATE_PARAMETERS,
        "LAYER_PARAMETERS": LAYER_PARAMETERS,
        "ZSL_PARAMETERS": ZSL_PARAMETERS,
        "INTERFACE_PARAMETERS": INTERFACE_PARAMETERS,
    }
)

Full Formula (Ni1)
Reduced Formula: Ni
abc   :   2.460000   2.460000   2.460000
angles:  60.000004  59.999994  60.000003
pbc   :       True       True       True
Sites (1)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Ni      0    0    0 

Full Formula (C2)
Reduced Formula: C
abc   :   2.467291   2.467291  20.000000
angles:  90.000000  90.000000 119.999986
pbc   :       True       True       True
Sites (2)
  #  SP           a         b    c
---  ----  --------  --------  ---
  0  C     0         0           0
  1  C     0.333333  0.666667    0 

Creating interfaces...


spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 4.750000
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 4.512500
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 4.286875
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 4.072531
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 3.868905
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 3.675459
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was 

### 3.2. Print out the interfaces and terminations

In [7]:
print(f'Found {len(terminations)} terminations')
for termination in terminations:
    print(f"Found {len(interfaces[termination])} interfaces for", termination, "termination")

Found 1 terminations
Found 233 interfaces for ('C_P6/mmm_2', 'Ni_R-3m_1') termination


## 4. Sort interfaces by strain

### 4.1. Sort all interfaces

In [8]:
# Could be "strain", "von_mises_strain", "mean_abs_strain"
strain_mode = "mean_abs_strain"

# Sort interfaces by the specified strain mode and number of sites
def sort_interfaces(interfaces, terminations):
    sorted_interfaces = {}
    for termination in terminations:
        sorted_interfaces[termination] = sorted(
            interfaces[termination], key=lambda x: (x[strain_mode], x["interface"].num_sites)
        )
    return sorted_interfaces


sorted_interfaces = sort_interfaces(interfaces, terminations)

### 4.2. Print out interfaces with lowest strain for each termination

In [9]:
for termination in terminations:
    print(f"Interface with lowest strain for termination {termination} (index 0):")
    first_interface = interfaces[termination][0]
    print("    strain:", first_interface[strain_mode] * 100, "%")
    print("    number of atoms:", first_interface["interface"].num_sites)

Interface with lowest strain for termination ('C_P6/mmm_2', 'Ni_R-3m_1') (index 0):
    strain: 0.06600000000000002 %
    number of atoms: 5


## 5. Plot the results

Plot the number of atoms vs strain. Adjust the parameters as needed.


In [10]:
import plotly.graph_objs as go
from collections import defaultdict

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}


def plot_strain_vs_atoms(sorted_interfaces, terminations, settings):
    # Create a mapping from termination to its index
    termination_to_index = {termination: i for i, termination in enumerate(terminations)}

    grouped_interfaces = defaultdict(list)
    for termination, interfaces in sorted_interfaces.items():
        for index, interface_data in enumerate(interfaces):
            strain_percentage = interface_data["mean_abs_strain"] * 100
            num_sites = interface_data["interface"].num_sites
            key = (strain_percentage, num_sites)
            grouped_interfaces[key].append((index, termination))
    
    data = []
    for (strain, num_sites), indices_and_terminations in grouped_interfaces.items():
        termination_indices = defaultdict(list)
        for index, termination in indices_and_terminations:
            termination_indices[termination].append(index)
        all_indices = [index for indices in termination_indices.values() for index in indices]
        index_range = f"{min(all_indices)}-{max(all_indices)}" if len(all_indices) > 1 else str(min(all_indices))
        
        hover_text = "<br>-----<br>".join(
             f"Termination: {termination}<br>Termination index: {termination_to_index[termination]}<br>Interfaces Index Range: {index_range}<br>Strain: {strain:.2f}%<br>Atoms: {num_sites}"
            for termination, indices in termination_indices.items()
        )
        trace = go.Scatter(
            x=[strain],
            y=[num_sites],
            text=[hover_text],
            mode="markers",
            hoverinfo="text",
            name=f"Indices: {index_range}",
        )
        data.append(trace)

    layout = go.Layout(
        xaxis=dict(title="Strain (%)", type=settings["X_SCALE"]),
        yaxis=dict(title="Number of atoms", type=settings["Y_SCALE"]),
        hovermode="closest",
        height=settings["HEIGHT"],
        legend_title_text="Interfaces Index Range",
    )
    fig = go.Figure(data=data, layout=layout)
    fig.show()



plot_strain_vs_atoms(sorted_interfaces, terminations, PLOT_SETTINGS)

for i, termination in enumerate(terminations):
    print(f"Termination {i}:", termination)

Termination 0: ('C_P6/mmm_2', 'Ni_R-3m_1')


## 6. Select the interface to pass outside this kernel

### 6.1. Select the interface with the desired termination and strain

The data in `sorted_interfaces` now contains an object with the following structure:

```json
{
    "('C_P6/mmm_2', 'Si_R-3m_1')": [
        { ...interface for ('C_P6/mmm_2', 'Si_R-3m_1') at index 0...},
        { ...interface for ('C_P6/mmm_2', 'Si_R-3m_1') at index 1...},
        ...
    ],
    "<termination at index 1>": [
        { ...interface for 'termination at index 1' at index 0...},
        { ...interface for 'termination at index 1' at index 1...},
        ...
    ]
}
```

Select the index for termination first, and for it - the index in the list of corresponding interfaces sorted by strain (index 0 has minimum strain).

In [11]:
termination_index = 0
number_of_interfaces_to_include = 1

termination = terminations[termination_index]

selected_interfaces = sorted_interfaces[termination][:number_of_interfaces_to_include]

### 6.2. Pass data to the outside runtime


In [12]:
from src.utils import from_pymatgen


materials = list(map(lambda interface_config: from_pymatgen(interface_config["interface"]), selected_interfaces))

materials

[{'name': 'Ni3 C2',
  'basis': {'elements': [{'id': 0, 'value': 'Ni'},
    {'id': 1, 'value': 'Ni'},
    {'id': 2, 'value': 'Ni'},
    {'id': 3, 'value': 'C'},
    {'id': 4, 'value': 'C'}],
   'coordinates': [{'id': 0,
     'value': [0.3333334309041891, 0.3333336930861107, 0.0]},
    {'id': 1,
     'value': [1.463562833148302e-07,
      5.396291662407293e-07,
      0.07434465064130956]},
    {'id': 2,
     'value': [0.6666668618083782, 0.6666673861722214, 0.14868930128261915]},
    {'id': 3, 'value': [0.0, 0.0, 0.2597298272022775]},
    {'id': 4, 'value': [0.3333330000000001, 0.333333, 0.2597298272022775]}],
   'units': 'crystal',
   'cell': [[2.4599995727812636, 0.0, 1.5063153013552154e-16],
    [1.2299995285143266, 2.130422844801467, 1.5063156049921537e-16],
    [0.0, 0.0, 27.017163104672278]],
   'constraints': []},
  'lattice': {'a': 2.4599995727812636,
   'b': 2.4600000686580152,
   'c': 27.017163104672278,
   'alpha': 90.0,
   'beta': 90.0,
   'gamma': 60.00001360342549,
   'unit

## Relaxation

In [13]:
interface = selected_interfaces[0]["interface"]
interface

Structure Summary
Lattice
    abc : 2.4599995727812636 2.4600000686580152 27.017163104672278
 angles : 90.0 90.0 60.00001360342549
 volume : 141.5926098507579
      A : 2.4599995727812636 0.0 1.5063153013552154e-16
      B : 1.2299995285143266 2.130422844801467 1.5063156049921537e-16
      C : 0.0 0.0 27.017163104672278
    pbc : True True True
PeriodicSite: Ni (1.23, 0.7101, 1.004e-16) [0.3333, 0.3333, 0.0]
PeriodicSite: Ni (1.024e-06, 1.15e-06, 2.009) [1.464e-07, 5.396e-07, 0.07434]
PeriodicSite: Ni (2.46, 1.42, 4.017) [0.6667, 0.6667, 0.1487]
PeriodicSite: C (0.0, 0.0, 7.017) [0.0, 0.0, 0.2597]
PeriodicSite: C (1.23, 0.7101, 7.017) [0.3333, 0.3333, 0.2597]

In [14]:
# Per https://github.com/materialsvirtuallab/matgl/blob/main/examples/Relaxations%20and%20Simulations%20using%20the%20M3GNet%20Universal%20Potential.ipynb

import matgl
from matgl.ext.ase import M3GNetCalculator, MolecularDynamics, Relaxer

pot = matgl.load_model("M3GNet-MP-2021.2.8-PES")

relaxer = Relaxer(potential=pot)
struct = interface
relax_results = relaxer.relax(struct, fmax=0.01)

# extract results
final_structure = relax_results["final_structure"]
final_energy = relax_results["trajectory"].energies[-1]
# print out the final relaxed structure and energy

print(final_structure)
print(f"The final energy is {float(final_energy):.3f} eV.")

/Users/mat3ra/code/green/api-examples/.venv-3.10/lib/python3.10/site-packages/matgl/apps/pes.py:60: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/Users/mat3ra/code/green/api-examples/.venv-3.10/lib/python3.10/site-packages/matgl/apps/pes.py:66: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/Users/mat3ra/code/green/api-examples/.venv-3.10/lib/python3.10/site-packages/matgl/apps/pes.py:67: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/Users/mat3ra/code/green/api-examples/.venv-3.10/lib/python3.10/site-packages/matgl/layers/_basis.py:121: 

Full Formula (Ni3 C2)
Reduced Formula: Ni3C2
abc   :   2.460151   2.460151  31.696931
angles:  89.999988  90.000007  60.000013
pbc   :       True       True       True
Sites (5)
  #  SP            a          b         c    bulk_equivalent  bulk_wyckoff    interface_label
---  ----  ---------  ---------  --------  -----------------  --------------  -----------------
  0  Ni     0.333333   0.333334  0.007396                  0  a               substrate
  1  Ni     0          1e-06     0.071393                  0  a               substrate
  2  Ni     0.666667   0.666667  0.135275                  0  a               substrate
  3  C     -0         -0         0.264215                  0  c               film
  4  C      0.333333   0.333333  0.264215                  0  c               film
The final energy is -34.621 eV.


In [15]:
final_structure

Structure Summary
Lattice
    abc : 2.460151333903277 2.4601506195880978 31.69693114999455
 angles : 89.99998780085411 90.00000687735196 60.00001257987962
 volume : 166.13893548105855
      A : 2.4601513339032497 -3.2738659652956474e-07 -1.594126913463637e-07
      B : 1.2300751255342108 2.130553040081757 2.827685773904575e-07
      C : -1.7507640005469454e-06 4.596760550776735e-06 31.69693114999417
    pbc : True True True
PeriodicSite: Ni (1.23, 0.7102, 0.2344) [0.3333, 0.3333, 0.007396]
PeriodicSite: Ni (1.576e-06, 1.55e-06, 2.263) [4.048e-07, 5.735e-07, 0.07139]
PeriodicSite: Ni (2.46, 1.42, 4.288) [0.6667, 0.6667, 0.1353]
PeriodicSite: C (-9.006e-07, 1.019e-06, 8.375) [-1.322e-07, -9.172e-08, 0.2642]
PeriodicSite: C (1.23, 0.7102, 8.375) [0.3333, 0.3333, 0.2642]

In [ ]:
from src.utils import from_pymatgen, poscar_to_ase

ase_structure = poscar_to_ase(final_structure.to(fmt="poscar"))
ase_interface = poscar_to_ase(interface.to(fmt="poscar"))

from ase.visualize import view

# view(ase_structure, viewer="x3d")


In [28]:
poscar_interface = interface.to(fmt="poscar")
poscar_final_structure = final_structure.to(fmt="poscar")
print(poscar_interface)
print(poscar_final_structure)

Ni3 C2
1.0
   2.4599995727812636    0.0000000000000000    0.0000000000000002
   1.2299995285143266    2.1304228448014668    0.0000000000000002
   0.0000000000000000    0.0000000000000000   27.0171631046722780
Ni C
3 2
direct
   0.3333334309041891    0.3333336930861107    0.0000000000000000 Ni
   0.0000001463562833    0.0000005396291662    0.0743446506413096 Ni
   0.6666668618083782    0.6666673861722214    0.1486893012826191 Ni
   0.0000000000000000    0.0000000000000000    0.2597298272022775 C
   0.3333330000000001    0.3333330000000000    0.2597298272022775 C

Ni3 C2
1.0
   2.4601513339032497   -0.0000003273865965   -0.0000001594126913
   1.2300751255342108    2.1305530400817569    0.0000002827685774
  -0.0000017507640005    0.0000045967605508   31.6969311499941711
Ni C
3 2
direct
   0.3333334142510999    0.3333337851806225    0.0073958603916488 Ni
   0.0000004048497147    0.0000005735050916    0.0713928639301646 Ni
   0.6666665051022833    0.6666670910516759    0.1352752271752413 Ni